# Web Scraping with BeautifulSoup

> Corresponding Recording: [Webscraping Lecture](https://vimeo.com/507104276/6ea87c957b)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

![soup](https://images.unsplash.com/photo-1542831371-29b0f74f9713?ixid=MXwxMjA3fDB8MHxzZWFyY2h8MXx8aHRtbHxlbnwwfHwwfA%3D%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=800&q=60)

## Agenda:  

SWBAT:

- Describe webscraping in the context of other data-gathering methods;
- Identify the basic structure of a webpage;
- Become familiar with the essential objects and critical methods of `BeautifulSoup`;
- Be able to scrape data from a live website.

## Context

We have already developed many ways of interacting with data.  We are able to:

* import .csv files (and other .csv-like objects)
* gather data from API's and interact with JSON objects
* query SQL databases

There is publicly available data all over the internet ripe for scraping, whether that be artist information data from wikipedia, song lyrics from songlyrics.com, or texts of famous books from Project Gutenberg. Below, we will learn how to navigate HTML and CSS to bring data into our local computers.

## The components of a web page

When we visit a web page, our web browser makes a GET request to a web server. The server then sends back files that tell our browser how to render the page for us. The files fall into a few main types:

- HTML — contain the main content of the page.
- CSS — add styling to make the page look nicer.
- JS — Javascript files add interactivity to web pages.
- Images — image formats, such as JPG and PNG allow web pages to show pictures.

After our browser receives all the files, it renders the page and displays it to us. There’s a lot that happens behind the scenes to render a page nicely, but we don’t need to worry about most of it when we’re web scraping.

### HTML


HyperText Markup Language (HTML) is a language that web pages are created in. HTML isn’t a programming language, like Python — instead, it’s a markup language that tells a browser how to layout content. 

Let’s take a quick tour through HTML so we know enough to scrape effectively. HTML consists of elements called tags. The most basic tag is the `<html>` tag. This tag tells the web browser that everything inside of it is HTML. We can make a simple HTML document just using this tag:

~~~
<html>
</html>
~~~
&#x2B07; Example cell with html in Markdown!

<html>
</html>

Right inside an html tag, we put two other tags, the head tag, and the body tag. The main content of the web page goes into the body tag. The head tag contains data about the title of the page, and other information that generally isn’t useful in web scraping:

~~~
<html>
    <head>
    </head>
    <body>
    </body>
</html>
~~~

We’ll now add our first content to the page, in the form of the p tag. The p tag defines a paragraph, and any text inside the tag is shown as a separate paragraph:
~~~
<html>
    <head>
    </head>
    <body>
          <p>
            Here's a paragraph of text!
        </p>
        <p>
            Here's a second paragraph of text!
        </p>
    </body>
</html>
~~~

<html>
    <head>
    </head>
    <body>
          <p>
            Here's a paragraph of text!
        </p>
        <p>
            Here's a second paragraph of text!
        </p>
    </body>
</html>

Tags have commonly used names that depend on their position in relation to other tags:

- **child** — a child is a tag inside another tag. So the two p tags above are both children of the body tag.
- **parent** — a parent is the tag another tag is inside. Above, the html tag is the parent of the body tag.
- **sibiling** — a sibiling is a tag that is nested inside the same parent as another tag. For example, head and body are siblings, since they’re both inside html. Both p tags are siblings, since they’re both inside body.

We can also add properties to HTML tags that change their behavior:

~~~
<html>
  <head></head>
  <body>
    <p>
      Here's a paragraph of text!
      <a href="https://www.dataquest.io">Learn Data Science Online</a>
    </p>
    <p>
      Here's a second paragraph of text!
      <a href="https://www.python.org">Python</a>        
    </p>
  </body>
</html>
~~~

<html>
    <head>
    </head>
    <body>
        <p>
            Here's a paragraph of text!
            <a href="https://www.dataquest.io">Learn Data Science Online</a>
        </p>
        <p>
            Here's a second paragraph of text!
            <a href="https://www.python.org">Python</a>        </p>
    </body></html>

In the above example, we added two a tags. a tags are links, and tell the browser to render a link to another web page. The href property of the tag determines where the link goes.

a and p are extremely common html tags. Here are a few others:

- *div*: indicates a division, or area, of the page.
- *b*: bolds any text inside.
- *i*: italicizes any text inside.
- *u*: underlines any text inside.
- *table*: creates a table.
- *form*: creates an input form.


For a full list of tags, look [here](https://developer.mozilla.org/en-US/docs/Web/HTML/Element).

<html>
    <b>bold</b> <br/>
    <i>italics</i> <br/>
    <u>underlining</u>
</html>

There are two special properties that give HTML elements names, and make them easier to interact with when we’re scraping: **class** and **id**. 

- One element can have multiple classes, and a class can be shared between elements. 
- Each element can only have one id, and an id can only be used once on a page. 
- Classes and ids are optional, and not all elements will have them.

We can add classes and ids to our example:

~~~
<html>
    <head>
    </head>
    <body>
        <p class="bold-paragraph">
            Here's a paragraph of text!
            <a href="https://www.dataquest.io" id="learn-link">Learn Data Science Online</a>
        </p>
        <p class="bold-paragraph extra-large">
            Here's a second paragraph of text!
            <a href="https://www.python.org" class="extra-large">Python</a>
        </p>
    </body>
</html>
~~~

<html>
    <head>
    </head>
    <body>
        <p class="bold-paragraph">
            Here's a paragraph of text!
            <a href="https://www.dataquest.io" id="learn-link">Learn Data Science Online</a>
        </p>
        <p class="bold-paragraph extra-large">
            Here's a second paragraph of text!
            <a href="https://www.python.org" class="extra-large">Python</a>
        </p>
    </body>
</html>

### Let's take a look at a sample web page from our local computer
- Open up example.com in your browser.
- Open up the inspector
    - Mac: cmd+option+c
    - Windows: ctrl+shift+c
- Click on the elements tab, and click on an element

## Make a sample webpage

Use [this site](https://htmledit.squarefree.com/) to see your page as you edit the underlying html.

## Webscraping with Python

### The requests library
The first thing we’ll need to do to scrape a web page is to download the page. We can download pages using the Python requests library (similar to interacting with APIs!).

In [2]:
req = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")

After running our request, we get a Response object. This object has a status_code property, which indicates if the page was downloaded successfully:

In [3]:
req.status_code

200

We can print out the HTML content of the page using the content property:

In [4]:
req.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

### Parsing a page with BeautifulSoup

We can use the BeautifulSoup library to parse this document, and extract the text from the `<p>` tag. 

In [5]:
soup = BeautifulSoup(req.content)
list(soup.children)

['html',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [6]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


As all the tags are nested, we can move through the structure one level at a time. We can first select all the elements at the top level of the page using the `children` property of `soup`.

In [7]:
list(soup.children)

['html',
 <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [8]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.Tag]

The `Tag` object allows us to navigate through an HTML document, and extract other tags and text. You can learn more about the various `BeautifulSoup` objects [here](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-objects).

We can now select the html tag and its children by taking the second item in the list:



In [9]:
html = list(soup.children)[1]
html

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<p>Here is some simple content for this page.</p>
</body>
</html>

Each item in the list returned by the `children` property is also a `BeautifulSoup` object, so we can also call the `children` method on `html`.

Now we can find the `children` inside the `html` tag:

In [10]:
list(html.children)

['\n',
 <head>
 <title>A simple example page</title>
 </head>,
 '\n',
 <body>
 <p>Here is some simple content for this page.</p>
 </body>,
 '\n']

As you can see above, there are two tags here, `head`, and `body`. We want to extract the text inside the `p` tag, so we’ll dive into the body:

In [11]:
body = list(html.children)[3]

We can now isolate the `p` tag:



In [12]:
p = list(body.children)[1]

Once we’ve isolated the tag, we can use the `get_text` method to extract all of the text inside the tag:

In [13]:
p.get_text

<bound method Tag.get_text of <p>Here is some simple content for this page.</p>>

### Finding all instances of a tag at once

If we want to extract a single tag, we can instead use the find_all method, which will find all the instances of a tag on a page.

In [14]:
soup = BeautifulSoup(req.content)
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

Note that `find_all` returns a list, so we’ll have to loop through, or use list indexing, it to extract text:

In [15]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

If you instead only want to find the first instance of a tag, you can use the `find` method, which will return a single `BeautifulSoup` object:

In [16]:
soup.find('p')

<p>Here is some simple content for this page.</p>

### Searching for tags by class and id


We introduced classes and ids earlier, but it probably wasn’t clear why they were useful. Classes and ids are used by CSS to determine which HTML elements to apply certain styles to. We can also use them when scraping to specify specific elements we want to scrape. Let's look at the following page:

~~~
<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <div>
            <p class="inner-text first-item" id="first">
                First paragraph.
            </p>
            <p class="inner-text">
                Second paragraph.
            </p>
        </div>
        <p class="outer-text first-item" id="second">
            <b>
                First outer paragraph.
            </b>
        </p>
        <p class="outer-text">
            <b>
                Second outer paragraph.
            </b>
        </p>
    </body>
</html>
~~~

In [17]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content)
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

Now, we can use the `find_all` method to search for items by class or by id. In the below example, we’ll search for any `p` tag that has the class `outer-text`:

In [18]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>,
 <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In the below example, we’ll look for any tag that has the class `outer-text`:



In [19]:
soup.find_all(class_="outer-text")[0]

<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>

We can also search for elements by `id`:


In [20]:
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

### More sophisticated webpages

In [21]:
url = 'https://forecast.weather.gov/MapClick.php?lat=41.8843&lon=-87.6324#.XdPlJUVKg6g'
request = requests.get(url)
soup = BeautifulSoup(request.content)

In [22]:
times = soup.find_all(class_='period-name')
times

[<p class="period-name">This<br/>Afternoon</p>,
 <p class="period-name">Tonight<br/><br/></p>,
 <p class="period-name">Friday<br/><br/></p>,
 <p class="period-name">Friday<br/>Night</p>,
 <p class="period-name">Saturday<br/><br/></p>,
 <p class="period-name">Saturday<br/>Night</p>,
 <p class="period-name">Sunday<br/><br/></p>,
 <p class="period-name">Sunday<br/>Night</p>,
 <p class="period-name">Monday<br/><br/></p>]

In [23]:
descs = soup.find_all(class_='short-desc')
descs

[<p class="short-desc">Sunny and<br/>Breezy</p>,
 <p class="short-desc">Mostly Clear</p>,
 <p class="short-desc">Mostly Sunny</p>,
 <p class="short-desc">Mostly Cloudy</p>,
 <p class="short-desc">Mostly Sunny</p>,
 <p class="short-desc">Mostly Cloudy</p>,
 <p class="short-desc">Mostly Cloudy</p>,
 <p class="short-desc">Breezy.<br/>Mostly Cloudy<br/>then Slight<br/>Chance<br/>Rain/Snow</p>,
 <p class="short-desc">Breezy.<br/>Chance<br/>Rain/Snow<br/>then Rain<br/>Likely</p>]

In [24]:
together = [(entry[0].text, entry[1].text) for entry in zip(times, descs)]
together

[('ThisAfternoon', 'Sunny andBreezy'),
 ('Tonight', 'Mostly Clear'),
 ('Friday', 'Mostly Sunny'),
 ('FridayNight', 'Mostly Cloudy'),
 ('Saturday', 'Mostly Sunny'),
 ('SaturdayNight', 'Mostly Cloudy'),
 ('Sunday', 'Mostly Cloudy'),
 ('SundayNight', 'Breezy.Mostly Cloudythen SlightChanceRain/Snow'),
 ('Monday', 'Breezy.ChanceRain/Snowthen RainLikely')]

### Pulling in a Table

*In general you'll need to examine the html code so that you can tell the BeautifulSoup parser what to look for!*

In [25]:
url = 'https://www.pro-football-reference.com/'

res = requests.get(url)
soup = BeautifulSoup(res.content)

In [26]:
tbodies = soup.find_all('tbody')
len(tbodies )

7

In [29]:
teams = []
table = soup.find('table', {'id': 'AFC'})

for row in table.find('tbody').find_all('tr'):
    try:
        team = {'name': row.find('th', {'data-stat': 'team'}).text,
               'wins': row.find('td', {'data-stat': 'wins'}).text,
               'losses': row.find('td', {'data-stat': 'losses'}).text,
               'ties': row.find('td', {'data-stat': 'ties'}).text}
        teams.append(team)
    except:
        pass

teams

[{'name': 'BUF*', 'wins': '13', 'losses': '3', 'ties': '0'},
 {'name': 'MIA', 'wins': '10', 'losses': '6', 'ties': '0'},
 {'name': 'NWE', 'wins': '7', 'losses': '9', 'ties': '0'},
 {'name': 'NYJ', 'wins': '2', 'losses': '14', 'ties': '0'},
 {'name': 'PIT*', 'wins': '12', 'losses': '4', 'ties': '0'},
 {'name': 'BAL+', 'wins': '11', 'losses': '5', 'ties': '0'},
 {'name': 'CLE+', 'wins': '11', 'losses': '5', 'ties': '0'},
 {'name': 'CIN', 'wins': '4', 'losses': '11', 'ties': '1'},
 {'name': 'TEN*', 'wins': '11', 'losses': '5', 'ties': '0'},
 {'name': 'IND+', 'wins': '11', 'losses': '5', 'ties': '0'},
 {'name': 'HOU', 'wins': '4', 'losses': '12', 'ties': '0'},
 {'name': 'JAX', 'wins': '1', 'losses': '15', 'ties': '0'},
 {'name': 'KAN*', 'wins': '14', 'losses': '2', 'ties': '0'},
 {'name': 'LVR', 'wins': '8', 'losses': '8', 'ties': '0'},
 {'name': 'LAC', 'wins': '7', 'losses': '9', 'ties': '0'},
 {'name': 'DEN', 'wins': '5', 'losses': '11', 'ties': '0'}]

## Combining our data into a Pandas DataFrame

We can now combine the data into a Pandas DataFrame and analyze it.

In order to do this, we’ll call the DataFrame class, and pass in each list of items that we have. We pass them in as part of a dictionary. Each dictionary key will become a column in the DataFrame, and each list will become the values in the column:

In [30]:
# Football data from the table in dictionary form (very easy!)

football = pd.DataFrame(teams)
football

,name,wins,losses,ties
0,BUF*,13,3,0
1,MIA,10,6,0
2,NWE,7,9,0
3,NYJ,2,14,0
4,PIT*,12,4,0
5,BAL+,11,5,0
6,CLE+,11,5,0
7,CIN,4,11,1
8,TEN*,11,5,0
9,IND+,11,5,0


In [31]:
# Weather data from the list of doubles

weather = pd.DataFrame(together,
                      columns=['time', 'description'])
weather

,time,description
0,ThisAfternoon,Sunny andBreezy
1,Tonight,Mostly Clear
2,Friday,Mostly Sunny
3,FridayNight,Mostly Cloudy
4,Saturday,Mostly Sunny
5,SaturdayNight,Mostly Cloudy
6,Sunday,Mostly Cloudy
7,SundayNight,Breezy.Mostly Cloudythen SlightChanceRain/Snow
8,Monday,Breezy.ChanceRain/Snowthen RainLikely
